<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/Stocks-Analysis/blob/main/Portfolio_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import yfinance as yf
import numpy as np


In [3]:
symbols = ['TSLA', 'AAPL', 'SPY', 'QQQ']

In [4]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

five_yrs_ago = datetime.now() - relativedelta(years=5)



data = yf.download(symbols, five_yrs_ago)['Adj Close']

[*********************100%***********************]  4 of 4 completed


In [5]:
print(data)

                  AAPL         QQQ         SPY        TSLA
Date                                                      
2017-09-19   37.661285  141.158951  229.387131   25.006666
2017-09-20   37.030167  140.723297  229.469711   24.927334
2017-09-21   36.394276  139.861557  228.854874   24.431999
2017-09-22   36.038383  139.726089  228.900772   23.406000
2017-09-25   35.720448  138.254456  228.432770   22.999332
...                ...         ...         ...         ...
2022-09-12  163.429993  310.739990  409.288727  304.420013
2022-09-13  153.839996  293.700012  391.491821  292.130005
2022-09-14  155.309998  296.029999  392.985687  302.609985
2022-09-15  152.369995  291.100006  388.524017  303.750000
2022-09-16  150.699997  289.320007  385.559998  303.350006

[1258 rows x 4 columns]


In [6]:
price_change = data.pct_change()
price_change['TSLA_net_position'] = price_change.TSLA.cumsum().shift().fillna(0)
price_change['AAPL_net_position'] = price_change.AAPL.cumsum().shift().fillna(0)
price_change['SPY_net_position'] = price_change.SPY.cumsum().shift().fillna(0)
price_change['QQQ_net_position'] = price_change.QQQ.cumsum().shift().fillna(0)

price_change['SPY_fixed'] = 100 + price_change['SPY_net_position'] * 100
price_change['TSLA_fixed'] = 100 + price_change['TSLA_net_position'] * 100
price_change['AAPL_fixed'] = 100 + price_change['AAPL_net_position'] * 100
price_change['QQQ_fixed'] = 100 + price_change['QQQ_net_position'] * 100

print(price_change) 

                AAPL       QQQ       SPY      TSLA  TSLA_net_position  \
Date                                                                    
2017-09-19       NaN       NaN       NaN       NaN           0.000000   
2017-09-20 -0.016758 -0.003086  0.000360 -0.003172           0.000000   
2017-09-21 -0.017172 -0.006124 -0.002679 -0.019871          -0.003172   
2017-09-22 -0.009779 -0.000969  0.000201 -0.041994          -0.023044   
2017-09-25 -0.008822 -0.010532 -0.002045 -0.017375          -0.065038   
...              ...       ...       ...       ...                ...   
2022-09-12  0.038508  0.011886  0.010748  0.015817           3.498299   
2022-09-13 -0.058680 -0.054837 -0.043483 -0.040372           3.514116   
2022-09-14  0.009555  0.007933  0.003816  0.035874           3.473744   
2022-09-15 -0.018930 -0.016654 -0.011353  0.003767           3.509618   
2022-09-16 -0.010960 -0.006115 -0.007629 -0.001317           3.513386   

            AAPL_net_position  SPY_net_position  Q

In [7]:
df = price_change.drop(price_change.index[0])

In [8]:
def cm_to_inch(value):
    return value/2.54


import matplotlib.pyplot as plt
import matplotlib.dates as mdates



plt.figure(figsize=(cm_to_inch(50), cm_to_inch(20)))
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=df.index,y=df['TSLA_fixed'], mode='lines', name='Tesla'))
fig.add_trace(go.Scatter(x=df.index, y=df['AAPL_fixed'],mode='lines',
                    name='Apple'))
fig.add_trace(go.Scatter(x=df.index, y=df['SPY_fixed'],mode='lines',
                    name='S&P 500'))
fig.add_trace(go.Scatter(x=df.index, y=df['QQQ_fixed'],mode='lines',
                    name='Nasdaq'))
fig.update_layout(title='Comparison of price return between S&P 500, Apple and Tesla in 1 year',
                   xaxis_title='Month',
                   yaxis_title='Price index (2017 = 100)')
fig.show()

<Figure size 1417.32x566.929 with 0 Axes>

In [9]:
plt.figure(figsize=(cm_to_inch(50), cm_to_inch(20)))

fig = go.Figure(data=go.Scatter(x=df.index,y=df['TSLA'], mode='markers', name='Tesla'))
fig.add_trace(go.Scatter(x=df.index, y=df['SPY'],mode='markers',
                    name='S&P 500'))
fig.add_trace(go.Scatter(x=df.index, y=df['AAPL'],mode='markers',
                    name='Apple'))
fig.update_layout(title='Comparison of price change per day between S&P 500, Apple and Tesla in 1 years',
                   xaxis_title='Month',
                   yaxis_title='Price change per day')
fig.show()

<Figure size 1417.32x566.929 with 0 Axes>

In [10]:
def covariance(x, y):
    # Finding the mean of the series x and y
    mean_x = sum(x)/float(len(x))
    mean_y = sum(y)/float(len(y))
    # Subtracting mean from the individual elements
    sub_x = [i - mean_x for i in x]
    sub_y = [i - mean_y for i in y]
    numerator = sum([sub_x[i]*sub_y[i] for i in range(len(sub_x))])
    denominator = len(x)-1
    cov = numerator/denominator
    return cov


covar_TSLA = covariance(df.TSLA, df.SPY)
covar_AAPL = covariance(df.AAPL, df.SPY)
variance = np.var(df.SPY)
beta_TSLA = round(covar_TSLA/variance,4)
beta_AAPL = round(covar_AAPL/variance,4)

In [11]:
print('Beta for Tesla: ', beta_TSLA)
print('Beta for Apple: ', beta_AAPL)

Beta for Tesla:  1.4689
Beta for Apple:  1.238


In [26]:
#Download multiple assets history
def download(tickers, start=None, end=None, actions=False, threads=True,
             group_by='column', auto_adjust=False, back_adjust=False,
             progress=True, period="max", show_errors=True, interval="1d", prepost=False,
             proxy=None, rounding=False, timeout=None, **kwargs):
    """Download yahoo tickers
    :Parameters:
        tickers : str, list
            List of tickers to download
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime.
            Default is 1900-01-01
        end: str
            Download end date string (YYYY-MM-DD) or _datetime.
            Default is now
        group_by : str
            Group by 'ticker' or 'column' (default)
        prepost : bool
            Include Pre and Post market data in results?
            Default is False
        auto_adjust: bool
            Adjust all OHLC automatically? Default is False
        actions: bool
            Download dividend + stock splits data. Default is False
        threads: bool / int
            How many threads to use for mass downloading. Default is True
        proxy: str
            Optional. Proxy server URL scheme. Default is None
        rounding: bool
            Optional. Round values to 2 decimal places?
        show_errors: bool
            Optional. Doesn't print errors if True
        timeout: None or float
            If not None stops waiting for a response after given number of
            seconds. (Can also be a fraction of a second e.g. 0.01)
    """

In [32]:
benchmark_ = ["QQQ",]
portfolio_ = ['AAPL', 'GOOG', 'AMZN', 'TSLA', 'NVDA']

start_date_ = "2017-09-19"
end_date_  = "2022-09-19"
number_of_scenarios = 10000

return_vector = []
risk_vector = []
distrib_vector = []

#Get Information from Benchmark and Portfolio
df = yf.download(benchmark_, period="5y")
df2 = yf.download(portfolio_, period="5y")

#Clean Rows with No Values on both Benchmark and Portfolio
df = df.dropna(axis=0)
df2 = df2.dropna(axis=0)

#Matching the Days
df = df[df.index.isin(df2.index)]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  5 of 5 completed


In [33]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-18,145.910004,146.369995,145.110001,145.550003,140.916916,24858200
2017-09-19,145.809998,146.009995,145.309998,145.800003,141.158951,20871200
2017-09-20,145.750000,145.910004,144.229996,145.350006,140.723267,34236200
2017-09-21,145.250000,145.259995,143.949997,144.460007,139.861618,28841800
2017-09-22,144.000000,144.529999,143.850006,144.320007,139.726059,21702400


In [35]:
# Analysis of Benchmark
benchmark_vector = np.array(df['Close'])

#Create our Daily Returns
benchmark_vector = np.diff(benchmark_vector)/benchmark_vector[1:]

#Select or Final Return and Risk
benchmark_return = np.average(benchmark_vector)
benchmark_risk = np.std(benchmark_vector)
# Analysis of Portfolio
portfolio_vector = np.array(df2['Close'])

#Create a loop for the number of scenarios we want:

for i in range(number_of_scenarios):
    #Create a random distribution that sums 1 
    # and is split by the number of stocks in the portfolio
    random_distribution = np.random.dirichlet(np.ones(len(portfolio_)),size=1)
    distrib_vector.append(random_distribution)
    
    #Find the Closing Price for everyday of the portfolio
    portfolio_matmul = np.matmul(random_distribution,portfolio_vector.T)
    
    #Calculate the daily return
    portfolio_matmul = np.diff(portfolio_matmul)/portfolio_matmul[:,1:]
    
    #Select or Final Return and Risk
    portfolio_return = np.average(portfolio_matmul, axis=1)
    portfolio_risk = np.std(portfolio_matmul, axis=1)
    
    #Add our Benchmark info to our lists
    return_vector.append(portfolio_return[0])
    risk_vector.append(portfolio_risk[0])
#Add our Benchmark info to our lists
return_vector.append(benchmark_return)
risk_vector.append(benchmark_risk)

In [36]:
#Create Risk Boundaries
delta_risk = 0.05
min_risk = np.min(risk_vector)
max_risk = risk_vector[0]*(1+delta_risk)
risk_gap = [min_risk, max_risk]